In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
from tqdm import tqdm
import copy

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Define the function f(x1, x2) - using a complex non-linear function
def f(x1, x2):
    """A non-linear function combining sinusoidal and quadratic terms"""
    return np.sin(3 * x1) + 0.5 * np.cos(4 * x2) + 0.5 * (x1**2 - x2**2)

# Define the threshold T for binary classification
T = 0.5

# Generate random samples in the unit square
def generate_samples(n_samples):
    """Generate random samples in the unit square [-1, 1] × [-1, 1]"""
    X = np.random.uniform(-1, 1, (n_samples, 2))
    x1, x2 = X[:, 0], X[:, 1]
    y_values = np.array([f(x1i, x2i) for x1i, x2i in zip(x1, x2)])
    y = (y_values >= T).astype(np.float32)
    return X, y

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        
        # Build sequential model with configurable hidden layers
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size
        
        # Output layer with sigmoid activation for binary classification
        layers.append(nn.Linear(prev_size, output_size))
        layers.append(nn.Sigmoid())
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)



# Function to compute predictions for a grid of points
def compute_predictions(model, n_points=100):
    """Compute model predictions on a grid"""
    x1_grid = np.linspace(-1, 1, n_points)
    x2_grid = np.linspace(-1, 1, n_points)
    X1, X2 = np.meshgrid(x1_grid, x2_grid)
    grid_points = np.vstack([X1.ravel(), X2.ravel()]).T
    
    # Get model predictions
    model.eval()
    with torch.no_grad():
        grid_points_tensor = torch.FloatTensor(grid_points)
        predictions = model(grid_points_tensor).numpy().flatten()
    
    # Reshape predictions for visualization
    pred_Z = predictions.reshape(X1.shape)
    
    return x1_grid, x2_grid, pred_Z

# Function to train model and save snapshots at specified intervals
def train_model_with_snapshots(X_train, y_train, save_epochs, max_epochs):
    """Train the model and save snapshots at specified epochs"""
    # Model parameters
    input_size = 2
    hidden_sizes = [20,]*5
    output_size = 1
    learning_rate = 0.001
    batch_size = 32
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
    
    # Create and initialize model
    model = MLP(input_size, hidden_sizes, output_size)
    
    # Loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create dataset and dataloader
    dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Dictionary to store model snapshots
    model_snapshots = {}
    
    # Save initial untrained model
    model_snapshots[0] = copy.deepcopy(model)
    
    # Track losses
    losses = []
    epoch_nums = []
    
    # Training loop
    for epoch in tqdm(range(1, max_epochs + 1)):
        model.train()
        epoch_loss = 0
        
        for batch_X, batch_y in dataloader:
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(dataloader)
        losses.append(avg_loss)
        epoch_nums.append(epoch)
        
        # Save model snapshot if it's in the save_epochs list
        if epoch in save_epochs:
            model_snapshots[epoch] = copy.deepcopy(model)
    
    # Create loss plot
    loss_fig = go.Figure()
    loss_fig.add_trace(go.Scatter(
        x=epoch_nums,
        y=losses,
        mode='lines+markers',
        name='Training Loss'
    ))
    
    loss_fig.update_layout(
        title=f'Training Loss (n={len(X_train)})',
        xaxis_title='Epoch',
        yaxis_title='Binary Cross-Entropy Loss',
        width=800,
        height=400,
    )
    
    return model_snapshots, loss_fig

# Function to visualize the true function with a 3D surface plot
def visualize_true_function():
    """Visualize the true function and its decision boundary using Plotly Surface plot"""
    n_points = 100
    x1_grid = np.linspace(-1, 1, n_points)
    x2_grid = np.linspace(-1, 1, n_points)
    X1, X2 = np.meshgrid(x1_grid, x2_grid)
    
    # Calculate function values
    true_Z = np.zeros((n_points, n_points))
    for i in range(n_points):
        for j in range(n_points):
            true_Z[i, j] = f(X1[i, j], X2[i, j])
    
    # Create 3D surface plot
    fig = go.Figure()
    
    # Add surface for function values
    fig.add_trace(go.Surface(
        z=true_Z,
        x=X1,
        y=X2,
        colorscale='Viridis',
        opacity=0.9,
        colorbar=dict(title='f(x1, x2) values'),
        name='Function Values'
    ))
    
    # Add a plane at z=T to visualize decision boundary
    x_plane = np.linspace(-1, 1, 2)
    y_plane = np.linspace(-1, 1, 2)
    X_plane, Y_plane = np.meshgrid(x_plane, y_plane)
    Z_plane = np.ones(X_plane.shape) * T
    
    fig.add_trace(go.Surface(
        z=Z_plane,
        x=X_plane,
        y=Y_plane,
        colorscale=[[0, 'rgba(255, 0, 0, 0.3)'], [1, 'rgba(255, 0, 0, 0.3)']],
        showscale=False,
        name='Decision Boundary'
    ))
    
    fig.update_layout(
        title=f'True Function f(x1, x2) with Decision Boundary (T = {T})',
        scene=dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='f(x1, x2)',
            camera=dict(
                eye=dict(x=1.87, y=0.88, z=1.5)
            ),
            annotations=[
                dict(
                    showarrow=False,
                    x=0.1,
                    y=0.1,
                    z=T,
                    text="Decision Boundary (T=0.5)",
                    xanchor="left",
                    xshift=10,
                    opacity=0.7
                )
            ]
        ),
        width=800,
        height=700,
    )
    
    # Add contours projection to the surface
    fig.update_traces(
        contours_z=dict(
            show=True, 
            usecolormap=True,
            highlightcolor="limegreen", 
            project_z=True
        )
    )
    
    return fig

# Create an interactive 3D visualization with epoch slider
def create_interactive_visualization(model_snapshots, X_train, y_train, n_samples):
    """Create an interactive 3D visualization with an epoch slider"""
    epochs = sorted(list(model_snapshots.keys()))
    
    # Pre-compute predictions for all epochs
    predictions_by_epoch = {}
    for epoch, model in model_snapshots.items():
        n_points = 50  # Reduced for performance
        x1_grid = np.linspace(-1, 1, n_points)
        x2_grid = np.linspace(-1, 1, n_points)
        X1, X2 = np.meshgrid(x1_grid, x2_grid)
        grid_points = np.vstack([X1.ravel(), X2.ravel()]).T
        
        # Get model predictions
        model.eval()
        with torch.no_grad():
            grid_points_tensor = torch.FloatTensor(grid_points)
            predictions = model(grid_points_tensor).numpy().flatten()
        
        # Reshape predictions for visualization
        pred_Z = predictions.reshape(X1.shape)
        predictions_by_epoch[epoch] = (X1, X2, pred_Z)
    
    # Create figure
    fig = go.Figure()
    
    # Initial epoch to display
    initial_epoch = epochs[0]
    X1, X2, pred_Z = predictions_by_epoch[initial_epoch]
    
    # Add surface for prediction probabilities
    surface = go.Surface(
        z=pred_Z,
        x=X1,
        y=X2,
        colorscale='Viridis',
        opacity=0.9,
        colorbar=dict(title='Probability'),
        name='Prediction Probabilities'
    )
    fig.add_trace(surface)
    
    # Add a plane at z=0.5 to visualize decision boundary
    x_plane = np.linspace(-1, 1, 2)
    y_plane = np.linspace(-1, 1, 2)
    X_plane, Y_plane = np.meshgrid(x_plane, y_plane)
    Z_plane = np.ones(X_plane.shape) * 0.5
    
    fig.add_trace(go.Surface(
        z=Z_plane,
        x=X_plane,
        y=Y_plane,
        colorscale=[[0, 'rgba(255, 0, 0, 0.3)'], [1, 'rgba(255, 0, 0, 0.3)']],
        showscale=False,
        name='Decision Boundary'
    ))
    
    # Split into positive and negative examples for different colors
    pos_samples = X_train[y_train.astype(bool)]
    neg_samples = X_train[~y_train.astype(bool)]
    
    # Add positive samples as 3D scatter points
    if len(pos_samples) > 0:
        pos_scatter = go.Scatter3d(
            x=pos_samples[:, 0],
            y=pos_samples[:, 1],
            z=np.ones(len(pos_samples)),  # Place points at z=1
            mode='markers',
            marker=dict(
                color='orange',
                size=4,
                line=dict(color='black', width=0.5)
            ),
            name='y = 1'
        )
        fig.add_trace(pos_scatter)
    
    # Add negative samples as 3D scatter points
    if len(neg_samples) > 0:
        neg_scatter = go.Scatter3d(
            x=neg_samples[:, 0],
            y=neg_samples[:, 1],
            z=np.zeros(len(neg_samples)),  # Place points at z=0
            mode='markers',
            marker=dict(
                color='blue',
                size=4,
                line=dict(color='black', width=0.5)
            ),
            name='y = 0'
        )
        fig.add_trace(neg_scatter)
    
    # Create slider steps
    steps = []
    for i, epoch in enumerate(epochs):
        X1, X2, pred_Z = predictions_by_epoch[epoch]
        
        step = dict(
            method="update",
            args=[
                # Update z values for the surface
                {"z": [pred_Z, Z_plane, None, None]},
                # Keep everything else the same
                {"title": f"MLP Predictions - n={n_samples}, Epoch {epoch}"}
            ],
            label=str(epoch)
        )
        steps.append(step)
    
    # Create slider
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Epoch: "},
        pad={"t": 50},
        steps=steps
    )]
    
    # Add contours projection to the surface
    fig.update_traces(
        contours_z=dict(
            show=True, 
            usecolormap=True,
            highlightcolor="limegreen", 
            project_z=True
        ),
        selector=dict(type='surface')
    )
    
    fig.update_layout(
        title=f"MLP Predictions - n={n_samples}, Epoch {initial_epoch}",
        scene=dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='Probability',
            camera=dict(
                eye=dict(x=1.87, y=0.88, z=1.5)
            )
        ),
        width=800,
        height=700,
        sliders=sliders
    )
    
    return fig

# Main function to run the experiment with interactive visualizations
if __name__ == "__main__":
    """Run the experiment with interactive epoch sliders for each sample size"""
    # First, visualize the true function
    print("Visualizing the true function and decision boundary:")
    true_func_fig = visualize_true_function()
    true_func_fig.show()
    
    # Sample sizes to test
    sample_sizes = [4000]
    
    # Set the maximum number of epochs
    max_epochs = 50
    
    # Define epochs to save (more frequent at the beginning, less frequent later)
    save_epochs = list(range(0, 21, 5)) + list(range(30, 101, 10)) + list(range(120, max_epochs + 1, 20))
    
    # Process each sample size
    for n_samples in sample_sizes:
        print(f"\n\nTraining with {n_samples} samples:")
        X_train, y_train = generate_samples(n_samples)
        
        # Train model and capture snapshots
        model_snapshots, loss_fig = train_model_with_snapshots(X_train, y_train, save_epochs, max_epochs)
        
        # Create interactive visualization
        interactive_fig = create_interactive_visualization(model_snapshots, X_train, y_train, n_samples)
        
        # Display the figures
        interactive_fig.show()
        loss_fig.show()



Visualizing the true function and decision boundary:




Training with 4000 samples:


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:08<00:00,  6.24it/s]


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
from tqdm import tqdm
import copy

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Define the function f(x1, x2) - using a complex non-linear function
def f(x1, x2):
    """A non-linear function combining sinusoidal and quadratic terms"""
    return np.sin(3 * x1) + 0.5 * np.cos(4 * x2) + 0.5 * (x1**2 - x2**2)

# Define the threshold T for binary classification
T = 0.5

# Generate random samples in the unit square
def generate_samples(n_samples):
    """Generate random samples in the unit square [-1, 1] × [-1, 1]"""
    X = np.random.uniform(-1, 1, (n_samples, 2))
    x1, x2 = X[:, 0], X[:, 1]
    y_values = np.array([f(x1i, x2i) for x1i, x2i in zip(x1, x2)])
    y = (y_values >= T).astype(np.float32)
    return X, y

# Define the MLP model with named modules
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        
        # Use ModuleDict for named modules
        self.layers = nn.ModuleDict()
        
        # Store layer names and dimensions for reference
        self.layer_info = {
            'input': {'name': 'Input Layer', 'size': input_size}
        }
        
        # Add hidden layers
        prev_size = input_size
        for i, hidden_size in enumerate(hidden_sizes):
            linear_name = f'linear{i+1}'
            relu_name = f'relu{i+1}'
            
            self.layers[linear_name] = nn.Linear(prev_size, hidden_size)
            self.layers[relu_name] = nn.ReLU()
            
            self.layer_info[linear_name] = {'name': f'Hidden Layer {i+1} (Linear)', 'size': hidden_size}
            self.layer_info[relu_name] = {'name': f'Hidden Layer {i+1} (ReLU)', 'size': hidden_size}
            
            prev_size = hidden_size
        
        # Add output layer
        output_linear = f'linear{len(hidden_sizes)+1}'
        self.layers[output_linear] = nn.Linear(prev_size, output_size)
        self.layers['sigmoid'] = nn.Sigmoid()
        
        self.layer_info[output_linear] = {'name': 'Output Layer (Linear)', 'size': output_size}
        self.layer_info['sigmoid'] = {'name': 'Output Layer (Sigmoid)', 'size': output_size}
        
        # Store layer names in order for forward pass
        self.layer_names = ['input'] + list(self.layers.keys())
    
    def forward(self, x):
        activations = {'input': x}
        
        for name in self.layer_names[1:]:  # Skip input layer
            x = self.layers[name](x)
            activations[name] = x
        
        return x, activations

# Function to train model and save snapshots at specified intervals
def train_model_with_snapshots(X_train, y_train, save_epochs, max_epochs):
    """Train the model and save snapshots at specified epochs"""
    # Model parameters
    input_size = 2
    hidden_sizes = [20, 10]
    output_size = 1
    learning_rate = 0.01
    batch_size = 32
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
    
    # Create and initialize model
    model = MLP(input_size, hidden_sizes, output_size)
    
    # Loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create dataset and dataloader
    dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Dictionary to store model snapshots
    model_snapshots = {}
    
    # Save initial untrained model
    model_snapshots[0] = copy.deepcopy(model)
    
    # Track losses
    losses = []
    epoch_nums = []
    
    # Training loop
    for epoch in tqdm(range(1, max_epochs + 1)):
        model.train()
        epoch_loss = 0
        
        for batch_X, batch_y in dataloader:
            # Forward pass - now returns both outputs and activations
            outputs, _ = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(dataloader)
        losses.append(avg_loss)
        epoch_nums.append(epoch)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{max_epochs}, Loss: {avg_loss:.4f}")
        
        # Save model snapshot if it's in the save_epochs list
        if epoch in save_epochs:
            model_snapshots[epoch] = copy.deepcopy(model)
    
    # Create loss plot
    loss_fig = go.Figure()
    loss_fig.add_trace(go.Scatter(
        x=epoch_nums,
        y=losses,
        mode='lines+markers',
        name='Training Loss'
    ))
    
    loss_fig.update_layout(
        title=f'Training Loss (n={len(X_train)})',
        xaxis_title='Epoch',
        yaxis_title='Binary Cross-Entropy Loss',
        width=800,
        height=400,
    )
    
    return model_snapshots, loss_fig

# Function to create scatter plot grid of hidden features
def plot_column_scatter_grid(data, y_labels, column_names=None, 
                            marker_size=5, opacity=0.7,
                            height=800, width=800,
                            title="Scatter Plot Matrix",
                            color_map=None):
    # Ensure data is a numpy array
    data = np.asarray(data)
    y_labels = np.asarray(y_labels)
    
    # Get number of columns
    n_cols = data.shape[1]
    
    # Generate default column names if not provided
    if column_names is None:
        column_names = [f"Feature {i+1}" for i in range(n_cols)]
    
    # Ensure we have the right number of column names
    if len(column_names) != n_cols:
        raise ValueError(f"Number of column names ({len(column_names)}) "
                         f"doesn't match number of columns in data ({n_cols})")
    
    # Ensure we have the right number of labels
    if len(y_labels) != data.shape[0]:
        raise ValueError(f"Number of labels ({len(y_labels)}) "
                         f"doesn't match number of samples in data ({data.shape[0]})")
    
    # Get unique classes
    unique_classes = np.unique(y_labels)
    
    # Create color map if not provided
    if color_map is None:
        default_colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880']
        color_map = {cls: default_colors[i % len(default_colors)] for i, cls in enumerate(unique_classes)}
    
    # Create subplot grid
    fig = make_subplots(
        rows=n_cols, 
        cols=n_cols,
        shared_xaxes=False, 
        shared_yaxes=False
    )
    
    # Loop through all pairs of columns
    for i in range(n_cols):
        for j in range(n_cols):
            # For each class, add a separate trace
            for cls in unique_classes:
                # Get indices for this class
                idx = y_labels == cls
                
                # Skip if no samples for this class
                if not np.any(idx):
                    continue
                
                # Extract data for this class and column pair
                x = data[idx, j]  # Column j will be on x-axis
                y = data[idx, i]  # Column i will be on y-axis
                
                # Add scatter plot trace for this class
                fig.add_trace(
                    go.Scatter(
                        x=x,
                        y=y,
                        mode='markers',
                        marker=dict(
                            size=marker_size,
                            opacity=opacity,
                            color=color_map[cls]
                        ),
                        name=f'Class {cls}',
                        # Only add legend for the first subplot
                        showlegend=(i==0 and j==0)
                    ),
                    row=i+1, col=j+1
                )
            
            # Add axis labels
            if i == n_cols-1:  # Bottom row
                fig.update_xaxes(title_text=column_names[j], row=i+1, col=j+1)
            if j == 0:  # First column
                fig.update_yaxes(title_text=column_names[i], row=i+1, col=j+1)
    
    # Update layout
    fig.update_layout(
        height=height, 
        width=width,
        title_text=title,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig

# Function to extract hidden layer activations for all epochs and layers
def extract_hidden_layers(model_snapshots, X_train):
    """Extract hidden layer activations for all model snapshots"""
    hidden_activations = {}
    
    for epoch, model in model_snapshots.items():
        print(f"Extracting activations for epoch {epoch}")
        
        x = torch.FloatTensor(X_train)
        _, activations_dict = model(x)  # Get activations directly from model forward pass
        
        # Convert tensor activations to numpy arrays
        activations_dict = {
            layer_name: act.detach().numpy() 
            for layer_name, act in activations_dict.items()
        }
        
        # Store activations for this epoch
        hidden_activations[epoch] = activations_dict
    
    return hidden_activations

# Function to sample features from activation data
def sample_features(data, max_features, rng=None):
    """Sample max_features from data if there are more features than max_features"""
    if rng is None:
        rng = np.random.RandomState(42)  # Use fixed seed for reproducibility
        
    n_features = data.shape[1]
    
    if n_features <= max_features:
        # Use all features if fewer than max_features
        return data, list(range(n_features))
    
    # Randomly sample feature indices without replacement
    feature_indices = rng.choice(n_features, max_features, replace=False)
    feature_indices.sort()  # Sort for consistency in visualization
    
    # Select sampled features
    sampled_data = data[:, feature_indices]
    
    return sampled_data, feature_indices

# Function to create interactive visualization with layer and epoch sliders
def create_dual_slider_visualization(hidden_activations, y_train, max_features=5):
    """Create an interactive visualization with sliders for both epoch and layer"""
    # Get available epochs and layers
    epochs = sorted(list(hidden_activations.keys()))
    
    # Get layer names (should be same for all epochs)
    layer_names = list(hidden_activations[epochs[0]].keys())
    
    # Create a random number generator for consistent feature sampling
    rng = np.random.RandomState(42)
    
    # Use first epoch, first layer data to determine dimensions
    initial_data = hidden_activations[epochs[0]][layer_names[0]]
    
    # Initial layer: use the first hidden layer if available
    initial_layer_name = layer_names[min(1, len(layer_names) - 1)]
    
    # Sample features and create feature names for each layer
    layer_features = {}
    for layer_name in layer_names:
        layer_data = hidden_activations[epochs[0]][layer_name]
        sampled_data, feature_indices = sample_features(layer_data, max_features, rng)
        layer_features[layer_name] = {
            'indices': feature_indices,
            'names': [f"Feature {idx+1}" for idx in feature_indices]
        }
    
    # Create human-readable layer names without relying on model.layer_info
    layer_display_names = {}
    for layer_name in layer_names:
        if layer_name == 'input':
            layer_display_names[layer_name] = 'Input Layer'
        elif 'linear' in layer_name:
            # Extract number from linear layer name, if any
            num = ''.join(filter(str.isdigit, layer_name))
            if num:
                layer_display_names[layer_name] = f'Hidden Layer {num} (Linear)'
            else:
                layer_display_names[layer_name] = 'Output Layer (Linear)'
        elif 'relu' in layer_name:
            # Extract number from relu layer name, if any
            num = ''.join(filter(str.isdigit, layer_name))
            if num:
                layer_display_names[layer_name] = f'Hidden Layer {num} (ReLU)'
            else:
                layer_display_names[layer_name] = 'ReLU Activation'
        elif 'sigmoid' in layer_name:
            layer_display_names[layer_name] = 'Output Layer (Sigmoid)'
        else:
            # Default case - capitalize the layer name
            layer_display_names[layer_name] = layer_name.capitalize()
    
    # Create initial visualization
    initial_data = hidden_activations[epochs[0]][initial_layer_name]
    sampled_data, _ = sample_features(initial_data, max_features, rng)
    feature_names = layer_features[initial_layer_name]['names']
    
    # Create base figure with initial data
    fig = plot_column_scatter_grid(
        sampled_data, 
        y_train,
        column_names=feature_names,
        title=f"{layer_display_names[initial_layer_name]} Features (Epoch {epochs[0]})"
    )
    
    # Create a lookup table for all possible scatter traces
    # This is to efficiently update the figure without recreating it each time
    trace_lookup = {}
    
    for epoch in tqdm(epochs, desc="Preparing visualization data"):
        for layer_name in layer_names:
            layer_data = hidden_activations[epoch][layer_name]
            feature_indices = layer_features[layer_name]['indices']
            
            # Sample features
            sampled_data, _ = sample_features(layer_data, max_features, rng)
            n_features = sampled_data.shape[1]
            
            # Skip if layer has no dimensions or no features
            if n_features == 0:
                continue
                
            # Create a dict to store scatter data for this epoch and layer
            trace_lookup[(epoch, layer_name)] = []
            
            # Loop through all pairs of features
            for i in range(n_features):
                for j in range(n_features):
                    # For each class
                    unique_classes = np.unique(y_train)
                    
                    for cls in unique_classes:
                        # Get indices for this class
                        idx = y_train == cls
                        
                        # Extract data for this class and feature pair
                        x = sampled_data[idx, j]
                        y = sampled_data[idx, i]
                        
                        # Store x and y data for this trace
                        trace_lookup[(epoch, layer_name)].append({
                            'row': i+1, 
                            'col': j+1, 
                            'x': x, 
                            'y': y,
                            'class': cls
                        })
    
    # Create frames for each combination of epoch and layer
    frames = []
    
    for epoch in epochs:
        for layer_name in layer_names:
            # Skip if this combination doesn't have data
            if (epoch, layer_name) not in trace_lookup:
                continue
                
            n_features = len(layer_features[layer_name]['indices'])
            frame_traces = []
            
            # Create traces for each subplot position
            for i in range(n_features):
                for j in range(n_features):
                    for cls_idx, cls in enumerate(np.unique(y_train)):
                        # Find the corresponding trace data
                        trace_idx = (i * n_features + j) * len(np.unique(y_train)) + cls_idx
                        
                        # Skip if index out of range
                        if trace_idx >= len(trace_lookup[(epoch, layer_name)]):
                            continue
                            
                        trace_data = trace_lookup[(epoch, layer_name)][trace_idx]
                        
                        # Create the trace for this frame
                        trace = go.Scatter(
                            x=trace_data['x'],
                            y=trace_data['y'],
                            mode='markers',
                            showlegend=False  # Legend is from base figure
                        )
                        
                        frame_traces.append(trace)
            
            # Create a frame for this epoch and layer
            frame = go.Frame(
                data=frame_traces,
                name=f"epoch{epoch}_layer{layer_name}",
                layout=go.Layout(title=f"{layer_display_names[layer_name]} Features (Epoch {epoch})")
            )
            frames.append(frame)
    
    # Add frames to the figure
    fig.frames = frames
    
    # Create sliders for epoch and layer
    epoch_steps = []
    for i, epoch in enumerate(epochs):
        step = {
            "args": [
                [f"epoch{epoch}_layer{initial_layer_name}"],  # First show initial layer with this epoch
                {"frame": {"duration": 300, "redraw": True}, "mode": "immediate"}
            ],
            "label": str(epoch),
            "method": "animate"
        }
        epoch_steps.append(step)
    
    layer_steps = []
    for layer_name in layer_names:
        step = {
            "args": [
                [f"epoch{epochs[0]}_layer{layer_name}"],  # First show initial epoch with this layer
                {"frame": {"duration": 300, "redraw": True}, "mode": "immediate"}
            ],
            "label": layer_display_names[layer_name],
            "method": "animate"
        }
        layer_steps.append(step)
    
    # Create sliders
    sliders = [
        {
            "active": 0,
            "yanchor": "top",
            "xanchor": "left",
            "currentvalue": {
                "font": {"size": 16},
                "prefix": "Epoch: ",
                "visible": True,
                "xanchor": "right"
            },
            "transition": {"duration": 300},
            "pad": {"b": 10, "t": 50},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": epoch_steps
        },
        {
            "active": layer_names.index(initial_layer_name),
            "yanchor": "top",
            "xanchor": "left",
            "currentvalue": {
                "font": {"size": 16},
                "prefix": "Layer: ",
                "visible": True,
                "xanchor": "right"
            },
            "transition": {"duration": 300},
            "pad": {"b": 10, "t": 120},
            "len": 0.9,
            "x": 0.1,
            "y": 0.1,
            "steps": layer_steps
        }
    ]
    
    # Update layout with sliders
    fig.update_layout(
        sliders=sliders,
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": 300, "redraw": True}, "fromcurrent": True}],
                        "label": "Play",
                        "method": "animate"
                    },
                    {
                        "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                        "label": "Pause",
                        "method": "animate"
                    }
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top"
            }
        ],
        # Extra margin at bottom for sliders
        margin=dict(l=50, r=50, t=100, b=150)
    )
    
    # Add interactive feature
    fig.update_layout(
        hovermode="closest",
        clickmode="event+select"
    )
    
    # Custom JavaScript for slider synchronization
    fig.add_annotation(
        text="Use sliders to explore features across epochs and layers",
        xref="paper", yref="paper",
        x=0.5, y=1.05,
        showarrow=False
    )
    
    return fig

# Main function to run the experiment with dual-slider visualization
def run_experiment_with_dual_sliders(max_features=5):
    """Run the experiment with dual sliders for epoch and layer"""
    # Generate training data
    n_samples = 200
    X_train, y_train = generate_samples(n_samples)
    
    # Set the maximum number of epochs
    max_epochs = 100
    
    # Define epochs to save
    save_epochs = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    
    # Train model and capture snapshots
    print("Training models and capturing snapshots...")
    model_snapshots, loss_fig = train_model_with_snapshots(X_train, y_train, save_epochs, max_epochs)
    
    # Extract hidden layer activations for all epochs and layers
    print("Extracting hidden layer activations...")
    hidden_activations = extract_hidden_layers(model_snapshots, X_train)
    
    # Create interactive visualization with dual sliders
    print("Creating interactive visualization...")
    interactive_fig = create_dual_slider_visualization(hidden_activations, y_train, max_features=max_features)
    
    # Display the figures
    loss_fig.show()
    interactive_fig.show()
    
    return model_snapshots, hidden_activations

if __name__ == "__main__":
    # Make max_features configurable
    max_features = 5  # Change this value to show more or fewer features
    global model_snapshots  # Make this available to the visualization function
    model_snapshots, hidden_activations = run_experiment_with_dual_sliders(max_features)

Training models and capturing snapshots...


 16%|████████████████                                                                                    | 16/100 [00:00<00:00, 153.00it/s]

Epoch 10/100, Loss: 0.2761
Epoch 20/100, Loss: 0.2112
Epoch 30/100, Loss: 0.1868


 34%|██████████████████████████████████                                                                  | 34/100 [00:00<00:00, 164.53it/s]

Epoch 40/100, Loss: 0.1718


 70%|██████████████████████████████████████████████████████████████████████                              | 70/100 [00:00<00:00, 169.97it/s]

Epoch 50/100, Loss: 0.1750
Epoch 60/100, Loss: 0.1400
Epoch 70/100, Loss: 0.1648
Epoch 80/100, Loss: 0.1169


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 168.88it/s]


Epoch 90/100, Loss: 0.1228
Epoch 100/100, Loss: 0.1044
Extracting hidden layer activations...
Extracting activations for epoch 0
Extracting activations for epoch 10
Extracting activations for epoch 20
Extracting activations for epoch 30
Extracting activations for epoch 40
Extracting activations for epoch 50
Extracting activations for epoch 60
Extracting activations for epoch 70
Extracting activations for epoch 80
Extracting activations for epoch 90
Extracting activations for epoch 100
Creating interactive visualization...


Preparing visualization data: 100%|███████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 464.74it/s]


In [6]:
import plotly.graph_objects as go
import networkx as nx
import plotly.io as pio
pio.renderers.default = 'iframe'

# Create a graph using NetworkX
G = nx.random_geometric_graph(20, 0.2)

# Get node positions
pos = nx.spring_layout(G)

# Create edges
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create nodes
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title=dict(  # Change here: use title dict instead of titleside
                text='Node Connections',
                side='right'  # You can specify the side here
            ),
            xanchor='left'
        )
    )
)

# Color nodes by number of connections
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(f'# of connections: {len(adjacencies[1])}')

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Network Graph',
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                ))

fig.show()